# Import Libraries

In [74]:
import pandas as pd
import numpy as np
import h5py
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import string as st
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import torch
import fasttext
from sklearn.decomposition import PCA, TruncatedSVD
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from collections import Counter
from sklearn.svm import SVC, LinearSVC

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

# Linguistic Modules

In [2]:
def preprocess_text(text):
    # Define interrogative words to KEEP
    interrogatives = {"what", "why", "how", "who", "where", "when", "which", "whom", "whose", "no", "not",
                    "very" ,"too" ,"too" ,"just", "if", "but", "however", "without", "like"}
    custom_stopwords = set(nlp.Defaults.stop_words)
    custom_stopwords -= interrogatives

    doc = nlp(text.lower().strip())  # Lowercase and remove whitespace
    
# Process tokens: lemmatize, filter stopwords/punct/numbers, keep interrogatives
    tokens = [
        token.lemma_ 
        for token in doc 
        if (
            (not token.is_stop or token.text in interrogatives) and  # Keep interrogatives
            not token.is_punct and token.is_alpha                                  # Remove punctuation
            # (token.is_alpha or token.like_num)                       # Keep words/numbers
        )
    ]

    return ' '.join(tokens)

In [3]:
def extract_linguistic_features(text):
    doc = nlp(text)
    features = {
        'num_verbs': len([token for token in doc if token.pos_ == 'VERB']),
        'num_nouns': len([token for token in doc if token.pos_ == 'NOUN']),
        'sentence_length': len(doc),
        'number_present': any(token.pos_ == 'NUM' for token in doc)
    }
    return features

# Pre-Processing

### Import Data

In [4]:
label_mapper = {
    'BT1' : 'knowledge',
    'BT2' : 'comprehension',
    'BT3' : 'application',
    'BT4' : 'analysis',
    'BT5' : 'synthesis',
    'BT6' : 'evaluation'
}

# Load dataset
df = pd.DataFrame()
for i in range(1,5):
    q_df = pd.read_csv(os.getcwd().replace('notebook' , 'dataset') + '/dataset' + str(i) + '.csv')
    df = pd.concat([df , q_df])

# Apply preprocessing
mask = df['label'].isin(label_mapper.keys())
df['label'] = df['label'].mask(mask, df['label'].map(label_mapper))

df['label'] = df['label'].str.lower()

df['processed_question'] = df['question'].apply(preprocess_text)

df['processed_question'] = [''.join(text) for text in df['processed_question']]

## Tokenize

### Vectorization

In [106]:
# token_pattern: words of 2+ word-chars, but not all digits
token_pattern = r"(?u)\b[a-zA-Z]{3,}\b"

vectorizer = TfidfVectorizer(min_df= 5 ,max_df=0.8 , ngram_range=(1,2), use_idf= True, norm= 'l2',
                             token_pattern= token_pattern)
tfidf = vectorizer.fit_transform(df['processed_question'])

# Convert TF-IDF sparse matrix to DataFrame with appropriate column names
tfidf_df = pd.DataFrame(tfidf.toarray(), columns= vectorizer.get_feature_names_out(), index=df.index)
tfidf_df.shape

(10708, 4198)

In [107]:
tfidf_df.head(5)

abc  abcd  able  absolute  absolute advantage  absorption  abstract  acceleration  accept  acceptance  access  access control  access resource  accommodate  accord  account  accounting  accuracy  accuracy predictive  accurate  achieve  achieve high  acid  acid residue  acoustic  act  action  activation  active  activism  activity  actual  actuator  adapt  adaptation  adaptive  adc  add  additive  additive manufacturing  address  address translation  adjustable  adjustable voltage  adjustment  admixture  adolescent  adopt  adoption  adsorption  adt  advanced  advancement  advantage  advantage disadvantage  advantage limitation  advantage production  advantage real  adverse  advertisement  advertiser  advertising  advertising campaign  advertising child  advice  aerial  aerial vehicle  aerodynamic  aerodynamic performance  aerospace  affect  age  agency  agent  aggregate  aggregate production  aggregation  aggregation process  agree  agreement  agriculture  aid  aid diagram  aim  \
0  0.0   0.0   0.0       0.0                 0.0         0.0       0.0           0.0     0.0         0.0     0.0             0.0              0.0          0.0     0.0      0.0    0.000000       0.0                  0.0       0.0      0.0           0.0   0.0           0.0       0.0  0.0     0.0         0.0     0.0       0.0       0.0     0.0       0.0    0.0         0.0       0.0  0.0  0.0       0.0                     0.0      0.0                  0.0         0.0                 0.0         0.0        0.0         0.0    0.0       0.0         0.0  0.0       0.0          0.0        0.0                     0.0                   0.0                   0.0             0.0      0.0            0.0         0.0          0.0                   0.0                0.0     0.0     0.0             0.0          0.0                      0.0        0.0     0.0  0.0     0.0    0.0        0.0                   0.0          0.0                  0.0    0.0        0.0          0.0  0.0          0.0  0.0   
1  0.0   0.0   0.0       0.0                 0.0         0.0       0.0           0.0     0.0         0.0     0.0             0.0              0.0          0.0     0.0      0.0    0.000000       0.0                  0.0       0.0      0.0           0.0   0.0           0.0       0.0  0.0     0.0         0.0     0.0       0.0       0.0     0.0       0.0    0.0         0.0       0.0  0.0  0.0       0.0                     0.0      0.0                  0.0         0.0                 0.0         0.0        0.0         0.0    0.0       0.0         0.0  0.0       0.0          0.0        0.0                     0.0                   0.0                   0.0             0.0      0.0            0.0         0.0          0.0                   0.0                0.0     0.0     0.0             0.0          0.0                      0.0        0.0     0.0  0.0     0.0    0.0        0.0                   0.0          0.0                  0.0    0.0        0.0          0.0  0.0          0.0  0.0   
2  0.0   0.0   0.0       0.0                 0.0         0.0       0.0           0.0     0.0         0.0     0.0             0.0              0.0          0.0     0.0      0.0    0.275685       0.0                  0.0       0.0      0.0           0.0   0.0           0.0       0.0  0.0     0.0         0.0     0.0       0.0       0.0     0.0       0.0    0.0         0.0       0.0  0.0  0.0       0.0                     0.0      0.0                  0.0         0.0                 0.0         0.0        0.0         0.0    0.0       0.0         0.0  0.0       0.0          0.0        0.0                     0.0                   0.0                   0.0             0.0      0.0            0.0         0.0          0.0                   0.0                0.0     0.0     0.0             0.0          0.0                      0.0        0.0     0.0  0.0     0.0    0.0        0.0                   0.0          0.0                  0.0    0.0        0.0          0.0  0.0          0.0  0.0   
3  0.0   0.

# Feature Extraction

### Adding Linguistic Features

Execute if linguistic features like count for verb,noun,etc needed

In [101]:
# Adding linguistic features
ling_features = df['question'].apply(extract_linguistic_features).apply(pd.Series)

# Concatenate DataFrames
token_df = pd.concat([ling_features, tfidf_df], axis=1)

token_df.columns = token_df.columns.map(str)

### Label Balancing

In [102]:
x , y = token_df , df['label']
x_resampled , y_resampled = SMOTETomek().fit_resample(x , y)

# Modelling

## Basic Machine Learning

In [103]:
x_train , x_test , y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=34 , stratify= y_resampled)

### Random Forest

In [108]:
params = {
    'n_estimators' : [2 , 5 , 10 , 100, 250 , 500],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 10 , 100 , 250 , 500]
}
param_search = GridSearchCV(estimator= RandomForestClassifier(), param_grid= params, cv=5, n_jobs= -1, verbose= 2)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END .....criterion=gini, max_depth=None, n_estimators=2; total time=   0.4s
[CV] END .....criterion=gini, max_depth=None, n_estimators=2; total time=   0.4s
[CV] END .....criterion=gini, max_depth=None, n_estimators=2; total time=   0.4s
[CV] END .....criterion=gini, max_depth=None, n_estimators=2; total time=   0.5s
[CV] END .....criterion=gini, max_depth=None, n_estimators=2; total time=   0.4s
[CV] END .....criterion=gini, max_depth=None, n_estimators=5; total time=   0.6s
[CV] END .....criterion=gini, max_depth=None, n_estimators=5; total time=   0.7s
[CV] END .....criterion=gini, max_depth=None, n_estimators=5; total time=   0.7s
[CV] END .....criterion=gini, max_depth=None, n_estimators=5; total time=   0.7s
[CV] END .....criterion=gini, max_depth=None, n_estimators=5; total time=   0.7s
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   1.3s
[CV] END ....criterion=gini, max_depth=None, n_

,mean_test_score,params
5,0.819104,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 500}"
4,0.818448,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 250}"
3,0.817313,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 100}"
2,0.789075,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 10}"
1,0.765433,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 5}"
0,0.676060,"{'criterion': 'gini', 'max_depth': None, 'n_estimators': 2}"
9,0.637373,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 100}"
8,0.570448,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}"
7,0.494030,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 5}"
6,0.402328,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 2}"


In [112]:
classifier = RandomForestClassifier(n_estimators= 500 , max_depth= None , criterion= 'gini')
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.94      0.87      0.91       564
  application       0.93      0.94      0.93       563
comprehension       0.87      0.85      0.86       554
   evaluation       0.81      0.81      0.81       559
    knowledge       0.77      0.85      0.80       552
    synthesis       0.82      0.80      0.81       558

     accuracy                           0.85      3350
    macro avg       0.86      0.85      0.85      3350
 weighted avg       0.86      0.85      0.85      3350



In [26]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)

print(classifier.predict(token_q))

['application']


### Decision Tree

In [ ]:
params = {
    'min_samples_split' : [2 , 5 , 10 , 50 , 100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 5 , 10 , 50 , 100 , 250 , 500]
}
param_search = GridSearchCV(estimator= DecisionTreeClassifier(), param_grid= params, cv=5, n_jobs= -1)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,mean_test_score,params
0,0.751050,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}"
1,0.747344,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 5}"
2,0.738196,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 10}"
3,0.711950,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 50}"
4,0.684508,"{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 100}"
5,0.340731,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2}"
7,0.340731,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 10}"
6,0.340551,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5}"
8,0.340551,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 50}"
9,0.340372,"{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 100}"


In [28]:
dt = DecisionTreeClassifier(criterion= 'gini', max_depth= None, min_samples_split = 2)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.85      0.84      0.85       563
  application       0.84      0.86      0.85       563
comprehension       0.83      0.81      0.82       554
   evaluation       0.72      0.75      0.73       559
    knowledge       0.74      0.75      0.75       548
    synthesis       0.74      0.70      0.72       559

     accuracy                           0.78      3346
    macro avg       0.78      0.78      0.78      3346
 weighted avg       0.78      0.78      0.78      3346



In [30]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)

print(dt.predict(token_q))

['analysis']


### XGBoost

In [11]:
y_mapper = {
    'knowledge' : 0,
    'comprehension' : 1,
    'application' : 2,
    'analysis' : 3,
    'synthesis' : 4,
    'evaluation' : 5
}

xgb_y = y_resampled.map(y_mapper)
xgb_y_train = y_train.map(y_mapper)
xgb_y_test = y_test.map(y_mapper)


In [20]:
params = {
    'max_depth' : [None , 250],
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1 , 0.3],
    'subsample': [0.8, 0.9]
}
param_search = GridSearchCV(estimator= XGBClassifier(objective='multi:softprob' , verbosity= 1), 
                            param_grid= params, cv=5, n_jobs= -1,return_train_score=True, verbose=2)

param_search.fit(x_resampled , xgb_y)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.8; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.8; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.8; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.8; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.9; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.9; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.8; total time= 1.7min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.9; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, subsample=0.9; total time= 1.6min
[CV] END learning_rate=0.01, max_depth=None, n_estimators=100, su

,mean_test_score,params
7,0.792534,"{'learning_rate': 0.01, 'max_depth': 250, 'n_estimators': 200, 'subsample': 0.9}"
6,0.791456,"{'learning_rate': 0.01, 'max_depth': 250, 'n_estimators': 200, 'subsample': 0.8}"
4,0.782657,"{'learning_rate': 0.01, 'max_depth': 250, 'n_estimators': 100, 'subsample': 0.8}"
5,0.782298,"{'learning_rate': 0.01, 'max_depth': 250, 'n_estimators': 100, 'subsample': 0.9}"
8,0.738424,"{'learning_rate': 0.1, 'max_depth': None, 'n_estimators': 100, 'subsample': 0.8}"
9,0.736867,"{'learning_rate': 0.1, 'max_depth': None, 'n_estimators': 100, 'subsample': 0.9}"
2,0.649174,"{'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 200, 'subsample': 0.8}"
3,0.649114,"{'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 200, 'subsample': 0.9}"
0,0.613198,"{'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 100, 'subsample': 0.8}"
1,0.610923,"{'learning_rate': 0.01, 'max_depth': None, 'n_estimators': 100, 'subsample': 0.9}"


In [ ]:
xgb = XGBClassifier(learning_rate=0.01, max_depth=None, n_estimators=200, subsample=0.8)
xgb.fit(x_train, xgb_y_train)
y_pred = xgb.predict(x_test)
print(classification_report(xgb_y_test , y_pred))

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [01:44:58] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1745056743506/work/src/learner.cc:738: 
Parameters: { "criterion", "min_samples_split" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.69      0.80      0.74       548
           1       0.82      0.76      0.79       553
           2       0.87      0.89      0.88       563
           3       0.91      0.80      0.85       563
           4       0.77      0.81      0.79       557
           5       0.81      0.76      0.79       558

    accuracy                           0.80      3342
   macro avg       0.81      0.80      0.81      3342
weighted avg       0.81      0.80      0.81      3342



In [ ]:
question = 'How many total disk access is needed to search a record using two level indexing?'
processed_q = preprocess_text(question)
tfidf_q = vectorizer.transform([processed_q])
tfidf_q = pd.DataFrame(tfidf_q.toarray(), columns=vectorizer.get_feature_names_out())



# Adding linguistic features
q_ling_features = pd.DataFrame([extract_linguistic_features(question)])

# Concatenate DataFrames
token_q = pd.concat([q_ling_features, tfidf_q], axis=1)
token_q = token_q.drop(columns= ['blooms_verb_present'])

print(xgb.predict(token_q))

### Linear SVM

In [105]:
params = {
    'penalty': ['l2'],
    'loss': ['squared_hinge' , 'hinge'],
    'dual': [True, False],
    'C': [0.1, 1, 10, 100]
}
param_search = GridSearchCV(estimator= LinearSVC(), param_grid= params, cv=5, n_jobs= -1, return_train_score=True, verbose= 2)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ..C=0.1, dual=False, loss=squared_hinge, penalty=l2; total time=   0.9s


/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ...C=0.1, dual=True, loss=squared_hinge, penalty=l2; total time=   6.8s


/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ...C=0.1, dual=True, loss=squared_hinge, penalty=l2; total time=   7.3s
[CV] END ..C=0.1, dual=False, loss=squared_hinge, penalty=l2; total time=   0.8s


...

/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ..C=100, dual=False, loss=squared_hinge, penalty=l2; total time=   1.1s
[CV] END ...........C=100, dual=True, loss=hinge, penalty=l2; total time=   8.1s


/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/nlp_eve/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV] END ...........C=100, dual=True, loss=hinge, penalty=l2; total time=   8.6s
[CV] END ..........C=100, dual=False, loss=hinge, penalty=l2; total time=   0.4s
[CV] END ...........C=100, dual=True, loss=hinge, penalty=l2; total time=   7.9s
[CV] END ...........C=100, dual=True, loss=hinge, penalty=l2; total time=   8.0s
[CV] END ..........C=100, dual=False, loss=hinge, penalty=l2; total time=   0.3s
[CV] END ..........C=100, dual=False, loss=hinge, penalty=l2; total time=   0.3s
[CV] END ..........C=100, dual=False, loss=hinge, penalty=l2; total time=   0.2s


,mean_test_score,params
6,0.785910,"{'C': 1, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l2'}"
4,0.766687,"{'C': 1, 'dual': True, 'loss': 'squared_hinge', 'penalty': 'l2'}"
5,0.759701,"{'C': 1, 'dual': True, 'loss': 'hinge', 'penalty': 'l2'}"
0,0.750269,"{'C': 0.1, 'dual': True, 'loss': 'squared_hinge', 'penalty': 'l2'}"
2,0.749970,"{'C': 0.1, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l2'}"
9,0.741194,"{'C': 10, 'dual': True, 'loss': 'hinge', 'penalty': 'l2'}"
8,0.739164,"{'C': 10, 'dual': True, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.711821,"{'C': 0.1, 'dual': True, 'loss': 'hinge', 'penalty': 'l2'}"
3,NaN,"{'C': 0.1, 'dual': False, 'loss': 'hinge', 'penalty': 'l2'}"
7,NaN,"{'C': 1, 'dual': False, 'loss': 'hinge', 'penalty': 'l2'}"


In [104]:
lsvm = LinearSVC(C= 1, dual= False, loss= 'squared_hinge', penalty= 'l2')
lsvm.fit(x_train, y_train)
y_pred = lsvm.predict(x_test)
print(classification_report(y_test , y_pred))

               precision    recall  f1-score   support

     analysis       0.84      0.86      0.85       564
  application       0.89      0.87      0.88       563
comprehension       0.79      0.77      0.78       554
   evaluation       0.81      0.81      0.81       559
    knowledge       0.75      0.73      0.74       552
    synthesis       0.79      0.82      0.80       558

     accuracy                           0.81      3350
    macro avg       0.81      0.81      0.81      3350
 weighted avg       0.81      0.81      0.81      3350



### SVM

In [ ]:
params = {
    'C': [0.001, 0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'degree': [1, 2, 4],
}
param_search = GridSearchCV(estimator= SVC(shrinking=False), param_grid= params, cv=5, n_jobs= -1, return_train_score=True, verbose= 2)

param_search.fit(x_resampled , y_resampled)

cv_results = pd.DataFrame(param_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].head(10).sort_values(by="mean_test_score", ascending=False)

cv_results 

Fitting 5 folds for each of 32 candidates, totalling 160 fits


ValueError: Invalid parameter 'dual' for estimator SVC(C=0.1, shrinking=False). Valid parameters are: ['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'].